In [17]:
import asyncio
import json
import os

from dotenv import load_dotenv
from hyperbrowser import AsyncHyperbrowser
from hyperbrowser.tools import WebsiteExtractTool
from openai import AsyncOpenAI
from openai.types.chat import (
    ChatCompletionMessageParam,
    ChatCompletionMessageToolCall,
    ChatCompletionToolMessageParam,
)

load_dotenv()

True

In [18]:
hb = AsyncHyperbrowser(api_key=os.getenv("HYPERBROWSER_API_KEY"))
llm = AsyncOpenAI()

In [19]:
async def handle_tool_call(
    tc: ChatCompletionMessageToolCall,
) -> ChatCompletionToolMessageParam:
    print(f"Handling tool call: {tc.function.name}")

    try:
        if (
            tc.function.name != WebsiteExtractTool.openai_tool_definition["function"]["name"]
        ):
            raise ValueError(f"Tool not found: {tc.function.name}")

        args = json.loads(tc.function.arguments)
        print(args)
        content = await WebsiteExtractTool.async_runnable(hb=hb, params=args)

        return {"role": "tool", "tool_call_id": tc.id, "content": content}

    except Exception as e:
        err_msg = f"Error handling tool call: {e}"
        print(err_msg)
        return {
            "role": "tool",
            "tool_call_id": tc.id,
            "content": err_msg,
            "is_error": True, #type: ignore
        }

In [20]:
async def agent_loop(messages: list[ChatCompletionMessageParam]) -> str:
    while True:
        response = await llm.chat.completions.create(
            messages=messages,
            model="gpt-4o",
            tools=[
                WebsiteExtractTool.openai_tool_definition,
            ],
            max_completion_tokens=8000,
        )

        choice = response.choices[0]

        # Append response to messages
        messages.append(choice.message) #type: ignore

        # Handle tool calls
        if choice.finish_reason == "tool_calls" and choice.message.tool_calls is not None:
            tool_result_messages = await asyncio.gather(
                *[handle_tool_call(tc) for tc in choice.message.tool_calls]
            )
            messages.extend(tool_result_messages)

        elif choice.finish_reason == "stop" and choice.message.content is not None:
            return choice.message.content

        else:
            print(choice)
            raise ValueError(f"Unhandled finish reason: {choice.finish_reason}")

In [21]:
SYSTEM_PROMPT = """
You are an expert coder. You have access to a 'extract_data' tool which can be used to get structured data from a webpage. 

This is the link to a piece of code {link}. You are required to find the input parameters, the output parameters, the template in which the code is to be provided, the language in which the code is to be written, the task to be performed, and the list of examples provided (in input and output format).
Once you have the information, you need to use those parameters to provide code that will adequately solve the given task. 
You are required to response with 
1. The task to be solved
2. The input parameters format
3. The output parameters format
4. The code template provided
5. The language in which the solution is required
6. The list of examples provided
7. Finally, and most importantly, the complete solution for the coding task given.
""".strip()

In [22]:
from typing import Coroutine, Any,Callable


def make_coding_agent(link_to_code: str)->Callable[..., Coroutine[Any, Any, str]]:
    # Popular documentation providers like Gitbook, Mintlify etc automatically generate a llms.txt file
    # for documentation sites hosted on their platforms.
    if not( link_to_code.startswith("http://") or link_to_code.startswith("https://")):
        link_to_code = f"https://{link_to_code}"

    sysprompt = SYSTEM_PROMPT.format(
        link=link_to_code,
    )

    async def solve_code(question: str) -> str:
        return await agent_loop([
            {"role": "system", "content": sysprompt},
            {"role": "user", "content": question},
        ])

    return solve_code

In [23]:
link_to_coding_task = "https://leetcode.com/problems/two-sum"

question = "Solve this coding problem"

agent = make_coding_agent(link_to_coding_task)

response = await agent(question)

print(response)

Handling tool call: extract_data
{'urls': ['https://leetcode.com/problems/two-sum'], 'prompt': 'Extract the task description, input/output parameters, code template, language, and examples for the problem on the webpage.', 'schema': '{"task": "string", "input_format": "string", "output_format": "string", "code_template": "string", "language": "string", "examples": [{"input": "string", "output": "string"}]}', 'max_links': 5}
Error handling tool call: schema - Invalid JSON schema - Status: 400 - Caused by HTTPStatusError: Client error '400 Bad Request' for url 'https://app.hyperbrowser.ai/api/extract'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400
Handling tool call: extract_data
{'urls': ['https://leetcode.com/problems/two-sum'], 'prompt': 'Extract the task description, input/output parameters, code template, language, and examples for the problem on the webpage.', 'schema': '{"type": "object", "properties": {"task": {"type": "string"}, "input_f

# ![Code Solver](assets/code_solver.png)
